In [1]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [2]:
## Caesar cypher, cyclically shift each letter by an amount k, specified by the key
## c = (p + k)mod 26   to encrypt (right shift)
## p = (c - k)mod 26   to decrypt (left shift)
## 
## there are only 26 possible distinct keys (from 0 to 25), this count includes the key that maps
## each letter onto itself, with a zero shift
def caesar(text, k):
    """
    text is a string, the message to encrypt or decrypt
    k is an int, the shift (can be positive or negative)
    """

    text = text.upper()
    k = k%26

    print(f"the text is {text}")
    print(f"the shift is {k}")

    result = ""
    for c in text:
        i = ALPHABET.index(c) + k
        i = i%26

        result = result + ALPHABET[i]

    return result


In [3]:
caesar("MEETMELATER", 23)

the text is MEETMELATER
the shift is 23


'JBBQJBIXQBO'

In [4]:
## monoalphabetic cypher: key is a permutation of alphabet, eg DKVQFIBJWPESCXHTMYAUOLRGZN 
## now there are 26! possible keys
## but the frequency distribution of english is retained
## 

def monoalphabetic(text, encrypt=True, KEY="DKVQFIBJWPESCXHTMYAUOLRGZN"):
    """
    text is a string, the message to encrypt, 
    KEY is a 26-character uppercase string, permutations of the ALPHABET

    encrypt is true if you want to encrypt, otherwise False
    
    """

    text = text.upper()
    print(f"the text is {text}")

    source_KEY = ALPHABET
    target_KEY = KEY

    ## invert the keys if you want to decrypt
    if encrypt==False:
        source_KEY, target_KEY = target_KEY, source_KEY 

    print(f"the KEY mappings are:\n{source_KEY}\n{target_KEY}")

    result = ""
    for c in text:
        i = source_KEY.index(c) 

        result = result + target_KEY[i]

    return result

In [5]:
monoalphabetic("AOVVFAA", encrypt=False)

#monoalphabetic("jfssh", source_KEY="DKVQFIBJWPESCXHTMYAUOLRGZN", target_KEY=ALPHABET )

the text is AOVVFAA
the KEY mappings are:
DKVQFIBJWPESCXHTMYAUOLRGZN
ABCDEFGHIJKLMNOPQRSTUVWXYZ


'SUCCESS'

In [6]:
def vigenere(text, KEY, encrypt=True):
    """
    text is a string, the message we want to encrypt
    KEY is a string of upper case letters representing the vigenere key 
    
    """
    text = text.upper()
    print(f"the text is {text}")
    print(f"the KEY is {KEY}")
    
    ## convert KEY to a list of shifts
    shifts = []
    for c in KEY:
        shifts.append(ALPHABET.index(c))   #nb index here works because ALPHABET does not repeat letters

    keylen = len(shifts)

    print(f"the shifts are {shifts}")
    
    ## now encrypt/decrypt text with this KEY
    result = ""

    for i in range(len(text)):
        
        shift = shifts[i%keylen]         # cyclically get the shift we want to apply to the ith character in the string

        if not encrypt:
            shift = -shift               # invert shift if decrypting
        
        c = text[i]             #get ith char in the string
        j = ALPHABET.index(c)   #retrieve its index in alphabet
        j_new = (j+shift)%26    # apply the shift
        result = result + ALPHABET[j_new]
    
    return result


In [7]:
vigenere("DRAGON", KEY="XO")

    


the text is DRAGON
the KEY is XO
the shifts are [23, 14]


'AFXULB'

In [8]:
def railfence(text, n):
    """
    text is a string, the message we want to encrypt,
    n an int, the number of rows for the railfence

    nb the letters must be placed in n rows going diagonally up and down 
    eg, for n=3, the row placements are
    0121 0121 0121 ....

    for n=5 the row placements are: 
    01234321 01234321 01234321...

    the pattern repeats every 2n-2 row placements
    """
    pattern = [i for i in range(n)] + [i for i in range(n-2, 0, -1)]  #count up from 0 to n-1 and back down from n-2 to 1

    print(pattern)

    rows = [""]*n      # a list of n strings, where we compile the rows

    for i in range(len(text)):
        r = pattern[ i%len(pattern) ]        # row where this letter should be appended
        rows[r] = rows[r] + text[i]

    result = "".join(rows)
    print(rows)
    print(result)
    


In [9]:
railfence("MEETMELATER", 3)

[0, 1, 2, 1]
['MMT', 'ETEAE', 'ELR']
MMTETEAEELR


In [10]:
n =2
pattern = [i for i in range(n)] + [i for i in range(n-2, 0, -1)]
print(pattern)

[0, 1]


In [20]:
import math
import numpy as np
def permutation(text, KEY):
    """
    text is a string, the message we want to encrypt
    KEY is a list of ints from 1 to len(KEY), eg [4, 2, 1, 3]

    we write out the text horizontally in a matrix of len(KEY) columns 
    then for the ciphertext take the colummns in the order specified by the KEY (after zero indexing it)

    eg for "MEETMELATER" and the above key of length 4:
  
     M E E T
     M E L A
     T E R 
    [3 1 0 2] = reordering specified by the key, the number below each column says its position in the sequence

    eg for this ciphertext the column corresponding to 0 ("ELR") goes first, then the one corresponding to 1 (MMT), etc
      ELREEETAMMT

    """
    n = len(KEY)                  # key length
                   
    print(f"the key is {KEY}, of length {n}")
    print(f"the plaintext is {text}")


    # adjust KEY so it is zero-indexed
    KEY = [val-1 for val in KEY]
    print(f"the zero-indexed KEY is {KEY}")

    # determine how much padding to add:
    # find remainder of division of text length by n
    # if nonzero pad with n - this amount
    pad = len(text)%n
    if pad != 0:
        text = text + "*"*(n-pad)

    print(text)

    mtrx = np.array( list(text) )
    print(mtrx)

    mtrx = mtrx.reshape(-1, n)   #reshape to however many number of rows x n columns
    print("\n key and original message matrix: ")
    print(KEY)
    print(mtrx)

    # construct the ciphertext by taking the columns in order

    cipher = ""
    print("ciphertext is")
    for i in range(n):           # go in order from 0 to n-1
        col = KEY.index(i)       # 0-indexed position in KEY of this ival
        letters = mtrx[:,col]    # retrieve list of letters at this column
        print(letters)
        cipher = cipher + "".join(letters)

    print(cipher)  
    ## THE BELOW COMMENTED CODE DOESN'T WORK, IT APPLIES A DIFFERENT
    ## LOGIC TO REORDER THE COLUMNS BASED ON THE KEY
    # # now rearrange the columns according to KEY
    # mtrx2 = mtrx[:,KEY]

    # print(f"\nwith reordered columns matrix is:")
    # print(mtrx2)

    # # now transpose to swop rows and columns
    # mtrx3 = mtrx2.transpose()
    # print(f"transposed is")
    # print(mtrx3)

    # # reshape to one row
    # cipher = mtrx3.reshape(1, -1)

    # print(f"\nreshaped to one row:")
    # print(cipher)
    # # change to list
    # cipher = cipher.tolist()
    # print(cipher[0])   #to get rid of nested outer list
    # s = "".join(cipher[0])
    # print(s)

    

In [21]:
permutation("MEETMELATER", [2,3,5,4,1])

the key is [2, 3, 5, 4, 1], of length 5
the plaintext is MEETMELATER
the zero-indexed KEY is [1, 2, 4, 3, 0]
MEETMELATER****
['M' 'E' 'E' 'T' 'M' 'E' 'L' 'A' 'T' 'E' 'R' '*' '*' '*' '*']

 key and original message matrix: 
[1, 2, 4, 3, 0]
[['M' 'E' 'E' 'T' 'M']
 ['E' 'L' 'A' 'T' 'E']
 ['R' '*' '*' '*' '*']]
ciphertext is
['M' 'E' '*']
['M' 'E' 'R']
['E' 'L' '*']
['T' 'T' '*']
['E' 'A' '*']
ME*MEREL*TT*EA*


In [22]:
text1 = caesar("MEETMELATER", 23)
print(f"text after ceaesar is {text1}")

text2 = permutation(text1, [5, 3, 1, 4, 2])


the text is MEETMELATER
the shift is 23
text after ceaesar is JBBQJBIXQBO
the key is [5, 3, 1, 4, 2], of length 5
the plaintext is JBBQJBIXQBO
the zero-indexed KEY is [4, 2, 0, 3, 1]
JBBQJBIXQBO****
['J' 'B' 'B' 'Q' 'J' 'B' 'I' 'X' 'Q' 'B' 'O' '*' '*' '*' '*']

 key and original message matrix: 
[4, 2, 0, 3, 1]
[['J' 'B' 'B' 'Q' 'J']
 ['B' 'I' 'X' 'Q' 'B']
 ['O' '*' '*' '*' '*']]
ciphertext is
['B' 'X' '*']
['J' 'B' '*']
['B' 'I' '*']
['Q' 'Q' '*']
['J' 'B' 'O']
BX*JB*BI*QQ*JBO


In [15]:
caesar("JBBQJBIXQBO", -23)

the text is JBBQJBIXQBO
the shift is 3


'MEETMELATER'